<a href="https://colab.research.google.com/github/Youchenjiang/AI-Final-Project/blob/change/%E4%BA%BA%E5%B7%A5%E6%99%BA%E6%85%A7%E6%9C%9F%E6%9C%AB_9_%E4%BF%AE%E6%94%B9%E5%8F%8A%E7%A7%BB%E9%99%A4%E9%83%A8%E5%88%86%E5%87%BD%E5%BC%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 人工智慧期末
TODO:
當你將排班從原本一個月28天（輪班）改為20天（正常班）後，的確有些懲罰函式需要調整或可以移除，因為工作模式的變化會影響排班的約束條件和目標。

以下是一些需要調整或移除的懲罰函式以及原因：

1. penalty_continuous_work():

原因： 這個函式是為了避免員工連續工作超過四天而設計的。 在輪班制度下，連續工作多天是常見的情況，因此需要這個懲罰函式。 但在正常班制度下，員工通常會有固定的休息日，連續工作超過四天的可能性很低，因此可以考慮移除或降低這個懲罰函式的權重。
2. check_labor_law():

原因： 這個函式是用來檢查排班是否符合勞基法的，例如員工一週最多工作6天，兩週最多工作11天等。 在輪班制度下，這些限制條件可能比較難滿足，但在正常班制度下，員工通常會有固定的休息日，更容易符合勞基法。 因此可以考慮調整這個函式，使其更符合正常班制度的規定，例如：
將一週最多工作6天的限制改為一週最多工作5天。
移除兩週最多工作11天的限制。
3. penalty_night_day():

原因： 這個函式是為了避免晚班-早班的安排，因為這樣會影響員工的休息。 在輪班制度下，晚班-早班的安排是常見的，因此需要這個懲罰函式。 但在正常班制度下，員工通常只會上日班，因此可以考慮移除這個懲罰函式。
4. calculate_shift_score():

原因： 這個函式是為了確保每個班別都有足夠的人員而設計的。 在輪班制度下，不同班別的人員需求可能不同，因此需要這個函式。 但在正常班制度下，所有員工都上日班，因此可以考慮簡化這個函式，只關注日班的人員需求。
建議：

移除或降低 penalty_continuous_work() 的權重。
調整 check_labor_law()，使其更符合正常班制度的規定。
移除 penalty_night_day()。
簡化 calculate_shift_score()，只關注日班的人員需求。
額外建議:

在調整懲罰函式後，需要重新測試演算法的性能，確保它仍然可以產生合理的排班結果。
可以根據實際情況，加入新的懲罰函式或調整現有懲罰函式的權重，以滿足特定的排班需求。

In [59]:
from statistics import *
from numpy import *
from random import *
from operator import itemgetter
from copy import deepcopy

CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.1
ITERATION_TIME = 500    #迭代次數
NUMBER_OF_GENETIC = 100  #基因數量
NUMBER_OF_WORKER = 16    #工作人數
WORK_DAY = 20

# 染色體集合
all_genetic = []
best_genetic = []
best_schedule_score = 100000000000

In [60]:
def calculate_shift_score(genetic, needToReachNumber, workType, showValue=False):
  score = 15
  shift_numbers = [0]*20

  # 記錄每天日班有多少人排班
  for i in range(len(genetic)):
    for j in range(WORK_DAY):
      if(genetic[i][j] == workType):
          shift_numbers[j] += 1

  #day_did_not_reach_number = sum(i < needToReachNumber for i in shift_numbers)
  day_did_not_reach_number = sum(1 for i in shift_numbers if i < needToReachNumber)
  if showValue:
    print('沒有達到',needToReachNumber,'人的天數', day_did_not_reach_number)
  '''shift_names = ["上課時段", "放學時段", "午休", "輪休"]
  if showValue:
    print("人員對應時間表安排:")
    for worker_id, schedule in enumerate(genetic):
      #print(f"人員 {worker_id + 1}: {schedule}")
      schedule_text = [shift_names[shift] for shift in schedule]
      print(f"人員 {worker_id + 1}: {schedule_text}")'''
  score = score * day_did_not_reach_number
  return score

In [61]:
# 判斷各種班別是否有達到上班人數標準，然後回傳一個分數
def calculate_total_shift_score(teamSchedule,showValue):
  # 0 上課時段 4人; 1 午休 2人; 2 放學時段 1人; 3 輪休
  inclassShiftScore = calculate_shift_score(teamSchedule, 4, 0, showValue)
  lunchbreakShiftScore = calculate_shift_score(teamSchedule, 2, 1, showValue)
  afterschoolShiftScore = calculate_shift_score(teamSchedule, 1, 2, showValue)

  score = inclassShiftScore + lunchbreakShiftScore + afterschoolShiftScore
  return score

In [62]:
def penalty_dayoff_work(genetic_list, number_of_worker, work_day, showValue):
  # 避免安排休假-工作-休假
  # 3 + (0,1,2) + 3
  penalty = 0
  count = 0
  for i in range(number_of_worker):
    for j in range(0,work_day-2):


      day1 = genetic_list[i][j]
      day2 = genetic_list[i][j+1]
      day3 = genetic_list[i][j+2]

      # 避免  3 - (0,1,2) - 3
      if (day1==3 and day3==3):
        if day2==3:
          pass
        else: # day2 == 0,1,2
          count += 1

  penalty = count * 1
  if(showValue):
    print('休假-工作-休假的次數：', count)
  return penalty


In [63]:
def penalty_night_day(genetic_list, number_of_worker, work_day, showValue):
  # 避免晚班-早班
  # (1,2) + 0

  count = 0
  for i in range(number_of_worker):
    for j in range(0,work_day-1):


      day1 = genetic_list[i][j]
      day2 = genetic_list[i][j+1]

      if (day1==1) or (day1==2):
        if day2==0:
          count += 1
  if(showValue):
    print('早班接晚班的次數：', count)
  penalty = count * 2
  return penalty

In [64]:
# 檢查是否有員工連續工作超過四天
def penalty_continuous_work(teamSchedule, showValue):
  penalty = 0
  for worker_schedule in teamSchedule:
    continuous_work_days = 0
    for day in worker_schedule:
      if day != 3:  # 非休假日
        continuous_work_days += 1
        if continuous_work_days > 4:
          penalty += 10  # 對連續工作超過四天進行懲罰
          break  # 一旦發現超過四天，就跳出內層迴圈
      else:  # 休假日
        continuous_work_days = 0
  if showValue:
    print('連續工作超過四天的次數：', penalty // 10)
  return penalty

In [65]:
def calculate_schedule_score(teamSchedule, showValue=False):
  day_shift = [] #日班
  later_shift = []
  grave_shift = []
  day_off = []
  for i in range(NUMBER_OF_WORKER):
    day_shift.append(len([i for i in teamSchedule[i] if i == 0])) # 日班
    later_shift.append(len([i for i in teamSchedule[i] if i == 1])) # 夜班
    grave_shift.append(len([i for i in teamSchedule[i] if i == 2])) # 大夜班
    day_off.append(len([i for i in teamSchedule[i] if i == 3])) # 休假次數

  ShiftNumberBasicScore = calculate_total_shift_score(teamSchedule,showValue)
  Penalty = penalty_dayoff_work(teamSchedule, NUMBER_OF_WORKER, WORK_DAY, showValue)+penalty_night_day(teamSchedule, NUMBER_OF_WORKER, WORK_DAY, showValue)
  continuous_work_penalty = penalty_continuous_work(teamSchedule, showValue)

  target_value = round(variance(day_shift)+variance(later_shift)+variance(grave_shift)+variance(day_off), 10) + ShiftNumberBasicScore + Penalty + continuous_work_penalty

  if showValue:
    print('Total variance: ', target_value )
    shift_names = ["上課時段", "放學時段", "午休", "輪休"]
    for day in range(WORK_DAY):
      print(f"日期 {day + 1}:")
      for shift in range(4):
        workers = [i + 1 for i, worker_schedule in enumerate(teamSchedule) if worker_schedule[day] == shift]
        print(f"  {shift_names[shift]}: {workers}")
  return target_value


In [66]:
def perform_crossover(): # 單點交配
  crossover_if = random()
  if( crossover_if > CROSSOVER_RATE): # 判斷是否要交配
    return
  else:
    # 隨機取兩個個體
    first = randint(0, NUMBER_OF_GENETIC-1)
    second = randint(0, NUMBER_OF_GENETIC-1)
    while(first==second):
      second = randint(0, NUMBER_OF_GENETIC-1)
    crossover_genetic_1 = all_genetic[first][:]
    crossover_genetic_2 = all_genetic[second][:]

    # 取得突變位置
    crossover_worker = int(random() * (NUMBER_OF_WORKER-1) ) # 取第幾個工人
    crossover_date = int(random() * (28-1) ) # 取工人第幾天的工作

    for i in range(crossover_date):
      temp = crossover_genetic_1[crossover_worker][i]
      crossover_genetic_1[crossover_worker][i] = crossover_genetic_2[crossover_worker][i]
      crossover_genetic_2[crossover_worker][i] = temp

    all_genetic[first] = deepcopy(crossover_genetic_1)
    all_genetic[second] = deepcopy(crossover_genetic_2)
  return


In [67]:
def perform_mutation(): # 每個染色體隨意找地方把一個值改成另一個
  global all_genetic
  muation_if = random()
  if( muation_if > MUTATION_RATE): # 判斷是否要突變
    return
  else:
    genetic_mutation = int(random() * (NUMBER_OF_GENETIC-1))
    worker_mutation = int(random() * (NUMBER_OF_WORKER-1))

    new_genetic = generate_worker_schedule()[:]
    all_genetic[genetic_mutation][worker_mutation] = new_genetic[:]
    return


In [68]:
# 生成母體必須符合：
# 1.員工一週最多工作6天(48小時)，最少2天-->休假數量最多=5(1周)
# 2.員工兩週最多工作11天
def generate_worker_schedule():
  each_worker = []
  work_type = 0
  workDay = 0 # 工作天數
  holiday = 0 # 休假數量
  twoWeeksWorkDay = 0 # 2個禮拜的工作天(第1、2周；第2、3周；第3、4周 三種可能)
  lastWeekWorkDay = 0

  for i in range(4):
    for j in range(7):

      if(holiday>=4):
        work_type = int(randint(0,2))
      elif(workDay>=6):
        work_type = int(randint(0,2))
        random_index = randrange(len(each_worker)-6, len(each_worker))
        each_worker[random_index] = 3
        holiday += 1
        workDay -= 1
      else: # random時給定一個比例
        type = [0, 0, 0, 0, 1, 1, 1, 2, 2, 3]
        work_type = choices( type )[0]

      if(work_type == 3):
        holiday += 1
      elif(work_type != 3):
        workDay +=1

      each_worker.append(work_type)

      if(i*7+j+1==14 or i*7+j+1==21 or i*7+j+1==28): # 第二週、第三週、第四週
        twoWeeksWorkDay = lastWeekWorkDay + workDay
        if(twoWeeksWorkDay > 11):
          for k in range(twoWeeksWorkDay - 11):
            random_index = randrange(len(each_worker)-6, len(each_worker))
            while(each_worker[random_index] == 3):
              random_index = randrange(len(each_worker)-6, len(each_worker))
            each_worker[random_index] = 3
            workDay -= 1
            holiday += 1

    lastWeekWorkDay = workDay
    workDay = 0
    holiday = 0
  return each_worker

In [69]:
def initialize_population():
  for i in range(NUMBER_OF_GENETIC):
    one_genetic = []
    for j in range(NUMBER_OF_WORKER):
      one_genetic.append(generate_worker_schedule()[:])
    all_genetic.append( one_genetic[:] )

    # 記錄最好的基因
    if(i == 0):
      best_genetic = one_genetic[:]
      best_schedule_score = calculate_schedule_score(best_genetic)
    elif(best_schedule_score > calculate_schedule_score(one_genetic) ):
      best_genetic = one_genetic[:]
      best_schedule_score = calculate_schedule_score(best_genetic)

In [70]:
def select_next_generation(number_of_generation):
  global all_genetic
  global best_schedule_score
  global best_genetic
  temp_all_genetic = []
  biggestTargetValue = -1
  population = []

  for j in range(NUMBER_OF_GENETIC):
    targetValue = calculate_schedule_score(all_genetic[j])
    population.append(targetValue)
    if(biggestTargetValue < targetValue):
      biggestTargetValue = targetValue

  # 如果全部基因都一樣沒有交配或突變就不用演進了
  different = True
  for i in range(1,len(population)):
    d = population[0]
    if(d!=population[i]):
      different = False

  if different:
    return

  for i in range(len(population)):
    population[i] = biggestTargetValue - population[i]

  for i in range(NUMBER_OF_GENETIC):
    x = choices( range(len(population)), population )
    temp_all_genetic.append(all_genetic[x[0]][:] )
  all_genetic = temp_all_genetic[:]

  for i in range(NUMBER_OF_GENETIC):
    targetValue = calculate_schedule_score(all_genetic[i])

    # 記錄最好的基因
    if(best_schedule_score > targetValue ):
      best_genetic = deepcopy(all_genetic[i])
      best_schedule_score = calculate_schedule_score(best_genetic)
  if(number_of_generation%10 == 0 or number_of_generation == 1):
    print('\n','======================= ITERATION ', number_of_generation ,' =======================')
    print('\n','----- Best Genetic Target Value: ', best_schedule_score, '-----')
    calculate_schedule_score(best_genetic, True)
  return

In [71]:
def run_genetic_algorithm():
  initialize_population() # 初始化: 建立所有基因
  for i in range(ITERATION_TIME):
    select_next_generation(i+1) # 選擇: 選擇下一代基因
    for j in range(7):
      perform_mutation() # 突變 => 突變一點太小了，這邊直接突變一個工人的全部
    for j in range(7):
      perform_crossover() # 交配
if __name__ == "__main__":
    run_genetic_algorithm()

<ipython-input-60-b6d621bdfd1f>:12: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  day_did_not_reach_number = sum(1 for i in shift_numbers if i < needToReachNumber)



 ======================= ITERATION  1  =======================

 ----- Best Genetic Target Value:  274.45 -----
沒有達到 4 人的天數 2
沒有達到 2 人的天數 0
沒有達到 1 人的天數 0
休假-工作-休假的次數： 9
早班接晚班的次數： 38
連續工作超過四天的次數： 15
Total variance:  274.45
日期 1:
  上課時段: [1, 3, 4, 6, 7]
  放學時段: [10, 11, 14, 15]
  午休: [9, 13]
  輪休: [2, 5, 8, 12, 16]
日期 2:
  上課時段: [1, 3, 4, 6, 9, 12]
  放學時段: [5, 7, 10, 13, 15, 16]
  午休: [8, 14]
  輪休: [2, 11]
日期 3:
  上課時段: [2, 4, 6, 13, 14]
  放學時段: [1, 3, 5, 7, 15, 16]
  午休: [9, 10, 11, 12]
  輪休: [8]
日期 4:
  上課時段: [4, 5, 8, 12]
  放學時段: [2, 9, 10, 11]
  午休: [1, 6, 14, 16]
  輪休: [3, 7, 13, 15]
日期 5:
  上課時段: [2, 4, 12, 14, 16]
  放學時段: [10, 11]
  午休: [1, 3, 5, 6, 7, 8]
  輪休: [9, 13, 15]
日期 6:
  上課時段: [2, 9, 11, 15]
  放學時段: [3, 8]
  午休: [7, 12, 16]
  輪休: [1, 4, 5, 6, 10, 13, 14]
日期 7:
  上課時段: [1, 2, 7, 10, 11, 13, 16]
  放學時段: [5, 14, 15]
  午休: [3, 4, 6, 8, 9]
  輪休: [12]
日期 8:
  上課時段: [2, 5, 7, 8, 10, 16]
  放學時段: [1, 12, 13, 15]
  午休: [6, 11, 14]
  輪休: [3, 4, 9]
日期 9:
  上課時段: [2, 4, 5, 9, 12, 16

KeyboardInterrupt: 